In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/07 16:29:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc=spark.sparkContext

In [4]:
import random
import datetime
from faker.providers.person.en import Provider

**Create a Employee table containing below columns. (1000 records) <br>
Employee Id, Employee Name,DOB, Job title , Job Start Date, Level ( bw 1 to 5)<br><br>
Create a Salary table Employee Id, Salary (bw 1L to 20L)**


In [5]:
sample_size=1000
start_id=1000
emp_id=list(range(start_id,start_id+sample_size))
emp_name=list(set(Provider.first_names))[0:sample_size]
dob=[datetime.date(random.randint(1982,2000), random.randint(1,12),random.randint(1,28)) for i in range(sample_size)]
job_title=random.choices(['Data Engineer','Sr Data Engineer','Data Scientist','Sr Data Scientist','Manager'],k=sample_size)
job_start_date=[datetime.date(random.randint(2012,2022), random.randint(1,12),random.randint(1,28)) for i in range(sample_size)]
level=[random.randrange(1, 6) for i in range(sample_size)]
salary=[random.randint(1, 20)*100000 for i in range(sample_size)]


In [6]:
df_emp=sc.parallelize(list(zip(emp_id,emp_name,dob,job_title,job_start_date,level)))\
            .toDF(("emp_id", "emp_name", "dob","job_title","job_start_date","level"))
df_emp.show(5)
df_sal=sc.parallelize(list(zip(emp_id,salary))).toDF(("emp_id","salary"))
df_sal.show(5)

+------+--------+----------+-----------------+--------------+-----+
|emp_id|emp_name|       dob|        job_title|job_start_date|level|
+------+--------+----------+-----------------+--------------+-----+
|  1000|    Bunk|1991-05-10|    Data Engineer|    2019-08-10|    1|
|  1001|  Dennis|1993-01-16| Sr Data Engineer|    2020-08-01|    5|
|  1002|   Gaven|1991-09-25| Sr Data Engineer|    2019-02-08|    4|
|  1003|  Aliyah|1993-05-15|Sr Data Scientist|    2016-11-06|    3|
|  1004|    Nola|1993-05-13|          Manager|    2022-09-08|    3|
+------+--------+----------+-----------------+--------------+-----+
only showing top 5 rows

+------+-------+
|emp_id| salary|
+------+-------+
|  1000|1500000|
|  1001|1100000|
|  1002| 700000|
|  1003| 400000|
|  1004|1200000|
+------+-------+
only showing top 5 rows



**Calculate years of experience and Age. (Use UDF )**

In [7]:
from pyspark.sql.functions import udf
@udf('int')
def my_date_udf(dt):
    from datetime import date
    curr = date.today()
    delta = (curr-dt).total_seconds()
    return round(int(delta/(60*60*24*365)),2)

In [8]:
from pyspark.sql.functions import col,datediff,current_date,floor

#Using Built-in datediff function
# df_emp.withColumn('Age',floor(datediff(current_date(),col('dob'))/365))\
#         .withColumn('years_exp',floor(datediff(current_date(),col('job_start_date'))/365)).show()

#Using UDF
df_1=df_emp.withColumn('age',my_date_udf(col('dob')))\
        .withColumn('years_exp',my_date_udf(col('job_start_date')))

df_1.show()



+------+--------+----------+-----------------+--------------+-----+---+---------+
|emp_id|emp_name|       dob|        job_title|job_start_date|level|age|years_exp|
+------+--------+----------+-----------------+--------------+-----+---+---------+
|  1000|    Bunk|1991-05-10|    Data Engineer|    2019-08-10|    1| 31|        3|
|  1001|  Dennis|1993-01-16| Sr Data Engineer|    2020-08-01|    5| 29|        2|
|  1002|   Gaven|1991-09-25| Sr Data Engineer|    2019-02-08|    4| 31|        3|
|  1003|  Aliyah|1993-05-15|Sr Data Scientist|    2016-11-06|    3| 29|        6|
|  1004|    Nola|1993-05-13|          Manager|    2022-09-08|    3| 29|        0|
|  1005|  Karina|1982-12-11|    Data Engineer|    2013-11-20|    2| 39|        8|
|  1006|    Chas|1993-03-21|   Data Scientist|    2022-03-25|    1| 29|        0|
|  1007|    Nova|1993-07-05| Sr Data Engineer|    2017-03-21|    5| 29|        5|
|  1008|    Nena|1988-12-01|          Manager|    2022-05-16|    4| 33|        0|
|  1009|   Lavar

**Create a flag EligibleForBonus if the years of experience is greater than 2 and salary is below 10L.**

In [9]:
from pyspark.sql.functions import col,when,lit

df_2=df_1.join(df_sal,on='emp_id',how='left')\
    .withColumn('EligibleForBonus',when( ((col('years_exp')>2) & (col('salary')<1000000)), lit('Y') )\
                                  .otherwise(lit('N')))
df_2.show()

+------+--------+----------+-----------------+--------------+-----+---+---------+-------+----------------+
|emp_id|emp_name|       dob|        job_title|job_start_date|level|age|years_exp| salary|EligibleForBonus|
+------+--------+----------+-----------------+--------------+-----+---+---------+-------+----------------+
|  1010|   Codey|1999-01-21| Sr Data Engineer|    2022-03-05|    4| 23|        0| 500000|               N|
|  1002|   Gaven|1991-09-25| Sr Data Engineer|    2019-02-08|    4| 31|        3| 700000|               Y|
|  1016|   Dayna|1999-03-25|Sr Data Scientist|    2021-10-27|    5| 23|        1|1400000|               N|
|  1012| Marques|1986-01-25|Sr Data Scientist|    2017-10-16|    4| 36|        5| 900000|               Y|
|  1009|   Lavar|1994-03-04|    Data Engineer|    2016-02-25|    2| 28|        6| 900000|               Y|
|  1000|    Bunk|1991-05-10|    Data Engineer|    2019-08-10|    1| 31|        3|1500000|               N|
|  1013|Berneice|1988-05-28|    Data 

**For employees eligible for Bonus, increment salary by 5% by each level i.e 5% for level 1 ,10% for level2 and so on. Save the results**

In [10]:
from pyspark.sql.functions import coalesce,isnull,col
lvls=df_2.select('level').distinct().rdd.flatMap(lambda x:x).collect()

print(lvls)
when_expr=[when( ((col('EligibleForBonus')=='Y') & (col('level')==i)),col('salary')+ col('salary')*0.05*i ) for i in lvls]
# when_expr 
df_3=df_2.select('*',coalesce(*when_expr).alias('new_sal'))\
        .withColumn('new_sal',when(isnull(col('new_sal')),col('salary')).otherwise(col('new_sal')).cast('int'))\

df_3.show()
df_3.write.csv('output/emp_2022_Q4_dtls',header=True,mode='overwrite')
                

[5, 1, 3, 2, 4]


+------+--------+----------+-----------------+--------------+-----+---+---------+-------+----------------+-------+
|emp_id|emp_name|       dob|        job_title|job_start_date|level|age|years_exp| salary|EligibleForBonus|new_sal|
+------+--------+----------+-----------------+--------------+-----+---+---------+-------+----------------+-------+
|  1010|   Codey|1999-01-21| Sr Data Engineer|    2022-03-05|    4| 23|        0| 500000|               N| 500000|
|  1002|   Gaven|1991-09-25| Sr Data Engineer|    2019-02-08|    4| 31|        3| 700000|               Y| 840000|
|  1016|   Dayna|1999-03-25|Sr Data Scientist|    2021-10-27|    5| 23|        1|1400000|               N|1400000|
|  1012| Marques|1986-01-25|Sr Data Scientist|    2017-10-16|    4| 36|        5| 900000|               Y|1080000|
|  1009|   Lavar|1994-03-04|    Data Engineer|    2016-02-25|    2| 28|        6| 900000|               Y| 990000|
|  1000|    Bunk|1991-05-10|    Data Engineer|    2019-08-10|    1| 31|        3

**Create a band of age group and categorize salary by age group.**


In [11]:
df_3.createOrReplaceTempView('emp_dtls')
sql_query="""
SELECT 
CASE WHEN age>=20 and age<25 then '20-25'
     WHEN age>=25 and age<30 then '25-30'
     WHEN age>=30 and age<35 then '30-35'
     WHEN age>=35 and age<40 then '35-40'
     WHEN age>=40 and age<45 then '40-45'
ELSE 'OTHER' END AS age_band
,SUM(salary) as sal
,COUNT(*) as cnt
FROM emp_dtls
GROUP BY 1
ORDER BY 1,2
"""
spark.sql(sql_query).show()

+--------+---------+---+
|age_band|      sal|cnt|
+--------+---------+---+
|   20-25|165100000|162|
|   25-30|296600000|272|
|   30-35|275700000|255|
|   35-40|281600000|266|
|   40-45| 47200000| 45|
+--------+---------+---+



**Avg Salary of the age group**

In [12]:
df_3.createOrReplaceTempView('emp_dtls')
sql_query="""
SELECT 
CASE WHEN age>=20 and age<25 then '20-25'
     WHEN age>=25 and age<30 then '25-30'
     WHEN age>=30 and age<35 then '30-35'
     WHEN age>=35 and age<40 then '35-40'
     WHEN age>=40 and age<45 then '40-45'
ELSE 'OTHER' END AS age_band
,CEIL(SUM(salary) /COUNT(*)) as avg_sal
FROM emp_dtls
GROUP BY 1
ORDER BY 1,2
"""
spark.sql(sql_query).show()



+--------+-------+
|age_band|avg_sal|
+--------+-------+
|   20-25|1019136|
|   25-30|1090442|
|   30-35|1081177|
|   35-40|1058647|
|   40-45|1048889|
+--------+-------+

